# Capstone Project
# Propective Community College locations in Chicago

Problem-----
a portion of the population of Chicago don't have access to an affordable college experience. This is because of the lack of community colleges in the less populated areas of Chicago.  




Background------
For many prospective students, college is very expensive. Many people consider community colleges as a way of starting their college careers. However there are some cluster of neighborhoods in Chicago which lack a nearbly community college. Students will have to take hours of self or public transportation to reach their community college which is probably located in another part of the city. The population around midway airport is one of the lowest in Chicago. These neighborhoods have residents among the lowest earning individuals.  

Access of education is a fundamental aspect that no communities should disregard. Through community colleges, students in the neighborhood will get access to more oppurtunities in the field. The overall poverty level can be further reduced as more people will get the oppurtunity to get college degrees and make earnings to support themselves and their families. In addition, the construction of the college will increase the number of jobs for workers and the opening of the college will increase the teaching positions available. In summary, the unemployment rate of the neighborhood should decrease. The audience who are the city officials who would license and fund the construction and operation of the college. This positive awarness of a new community college will better improve the education and living conditions of many residents. The officials are looking for ways to improve the city's education system so this new plan of creating colleges in low populated areas might improve the overall living condition in the neighborhood.

Data ----------  the data collected is the locations of the nearby community colleges around one of less populated areas in Chicago. The data includes the distance from the center of the neighborhood and the type of institution. The data that will be extracted is the distance, latitude and longitude, name and categories. 

The data will be used to create a cluster using Kmeans to determine the two closest location from the neighborhood center to build two new community colleges. The objective is to create a new cluster around the west side of the airport at a distance of less than 2000 meters. 

The data will be collected from foursquare. First observations are that the many of the nearby colleges are located more than 13km way and not many clusters on the west side of the airport. This means that a college is needed on the west side of the airport. 


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = 'SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF' # your Foursquare ID
CLIENT_SECRET = 'QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF
CLIENT_SECRET:QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE


In [3]:
df = pd.read_csv("https://data.cityofchicago.org/api/views/b7zw-zvm2/rows.csv?accessType=DOWNLOAD")

df.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [4]:
df1 = df[["COMMUNITY AREA NAME", "HARDSHIP INDEX"]]

df1["HARDSHIP INDEX"] = df1[df1["HARDSHIP INDEX"] > 80]["HARDSHIP INDEX"]

df1.dropna()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,COMMUNITY AREA NAME,HARDSHIP INDEX
22,Humboldt park,85.0
25,West Garfield Park,92.0
26,East Garfield Park,83.0
28,North Lawndale,87.0
29,South Lawndale,96.0
33,Armour Square,82.0
36,Fuller Park,97.0
39,Washington Park,88.0
53,Riverdale,98.0
57,Brighton Park,84.0


In [5]:
address = 'Fuller Park Chicago, IL 60609'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8180891 -87.6325508


In [6]:
search_query = 'Community college'
radius = 20000
print(search_query + ' .... OK!')

Community college .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SSWCFOUOB0WOECC1NNHRZQAVPMI3EY4UOC1LWS4YFJVTKFEF&client_secret=QVGIRPWYW33KOE403CS0K3C2BSED1ZOCRVJHIX0GICG434QE&ll=41.8180891,-87.6325508&v=20180604&query=Community college&radius=20000&limit=30'

In [8]:
results = requests.get(url).json()
#results

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


dataframe_filtered = dataframe_filtered[dataframe_filtered.categories == 'Community College']
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Moraine Valley Community College,Community College,9000 W College Pkwy,US,Palos Hills,United States,NaN,22074,"[9000 W College Pkwy, Palos Hills, IL 60465, U...","[{'label': 'display', 'lat': 41.69194156067703...",41.691942,-87.837641,NaN,60465,IL,49e0bafef964a52064611fe3
1,Oakton Community College,Community College,7701 Lincoln Ave,US,Skokie,United States,at Skokie Blvd,24576,"[7701 Lincoln Ave (at Skokie Blvd), Skokie, IL...","[{'label': 'display', 'lat': 42.02095077792238...",42.020951,-87.749617,NaN,60077,IL,4adc63e1f964a520fe2b21e3
6,Moraine Valley Community College Southwest Edu...,Community College,NaN,US,Tinley Park,United States,NaN,22077,"[Tinley Park, IL 60477, United States]","[{'label': 'display', 'lat': 41.69330344761157...",41.693303,-87.839183,NaN,60477,IL,4eb3f7f2991165b761c201f5
19,Richard J. Daley College,Community College,7500 S Pulaski Rd,US,Chicago,United States,NaN,10264,"[7500 S Pulaski Rd, Chicago, IL 60652, United ...","[{'label': 'display', 'lat': 41.75605576300383...",41.756056,-87.724051,NaN,60652,IL,4b71f4e2f964a52084662de3
22,Malcolm X College,Community College,1900 W Van Buren St,US,Chicago,United States,at S Damen Ave.,7568,"[1900 W Van Buren St (at S Damen Ave.), Chicag...","[{'label': 'display', 'lat': 41.87788496998021...",41.877885,-87.675990,NaN,60612,IL,4b61da40f964a52014272ae3
27,Everest College,Community College,247 S State St #400,US,Chicago,United States,btwn W Adams St & E Jackson St,1082,[247 S State St #400 (btwn W Adams St & E Jack...,"[{'label': 'display', 'lat': 41.808377, 'lng':...",41.808377,-87.631822,NaN,60604,IL,4b8830bef964a5203ae631e3


In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [19]:
average_long = dataframe_filtered["lng"].mean()
average_lat = dataframe_filtered["lat"].mean()


41.808085586532506
-87.74305060036436


In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

folium.features.CircleMarker(
    [average_lat, average_long],
    radius=20,
    color='green',
    popup='new College',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)




# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map